# 인공지능 코드 작성 (스팸 문자 판별기)
---


### *0. 파일 불러오기*

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


---
## *1. 데이터 전처리*

In [ ]:
# 0.라이브러리 불러오기
import pandas as pd
import re

# 1.데이터 불러오기
df = pd.read_csv('spam.csv', encoding='latin-1')

# 2.불필요한 컬럼 제거
df = df[['v1', 'v2']]
df.columns = ['label', 'text']

# 3.결측치 확인 및 제거
df.dropna(inplace=True)

# 4.라벨 정수로 변환
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# 5. 텍스트 전처리 함수 정의
def clean_text(text):
    text = re.sub(r'http\S+|www\S+', '[URL]', text)
    text = re.sub(r'\d{2,4}[-\s]?\d{3,4}[-\s]?\d{4}', '[PHONE]', text)

    # 이메일 주소 대체
    text = re.sub(r'\S+@\S+', '[EMAIL]', text)

    # 강조용 특수문자 외 제거
    text = re.sub(r'[^A-Za-z0-9!$%@#\s]', '', text)

    # 불필요한 공백 정리
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 6. 전처리 적용
df['clean_text'] = df['text'].apply(clean_text)

# 7. 중복 제거 (같은 문장이 여러 번 들어간 경우)
df.drop_duplicates(subset='clean_text', inplace=True)

# 8. 전처리 결과 확인
print("데이터 개수:", len(df))
print(df.sample(5))



데이터 개수: 5113
      label                                               text  \
1581      0  Hhahhaahahah rofl wtf nig was leonardo in your...   
4941      0  I'm eatin now lor, but goin back to work soon....   
4592      0  It's ok i wun b angry. Msg u aft i come home t...   
1545      0  Good afternoon, my love ! Any job prospects ? ...   
2740      0                                    Nothing. Can...   

                                             clean_text  
1581  Hhahhaahahah rofl wtf nig was leonardo in your...  
4941  Im eatin now lor but goin back to work soon E ...  
4592  Its ok i wun b angry Msg u aft i come home ton...  
1545  Good afternoon my love ! Any job prospects Are...  
2740                                        Nothing Can  


---
### *2. TfidVectorizer 사용하기*

In [ ]:
# 라이브러리 불러오기
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from collections import Counter

# 1. 데이터 분리
X = df['clean_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("원본 데이터 비율:", Counter(y_train))

# 2. TF-IDF 벡터화
vectorizer = TfidfVectorizer(
    max_features=5113,
    ngram_range=(1, 2),
    stop_words='english'
)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF 완료:", X_train_tfidf.shape)

# 3. SMOTE 적용 (학습용 데이터에만)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

print("\nSMOTE 적용 후 데이터 비율:", Counter(y_train_resampled))
print("SMOTE 적용 전:", X_train_tfidf.shape, "→ 적용 후:", X_train_resampled.shape)


원본 데이터 비율: Counter({0: 3600, 1: 490})
TF-IDF 완료: (4090, 5000)

SMOTE 적용 후 데이터 비율: Counter({1: 3600, 0: 3600})
SMOTE 적용 전: (4090, 5000) → 적용 후: (7200, 5000)
